<a href="https://colab.research.google.com/github/rgurve/NLP/blob/main/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 136kB/s 
     |████████████████████████████████| 6.8MB 45.7MB/s 
     |████████████████████████████████| 983kB 53.1MB/s 
     |████████████████████████████████| 266kB 60.2MB/s 
     |████████████████████████████████| 1.3MB 47.8MB/s 
     |████████████████████████████████| 1.1MB 52.0MB/s 
     |████████████████████████████████| 471kB 54.8MB/s 
     |████████████████████████████████| 2.9MB 48.7MB/s 
     |████████████████████████████████| 890kB 51.8MB/s 
  Created wheel for ktrain: filename=ktrain-0.25.3-cp36-none-any.whl size=25276312 sha256=155eed4eb7f4c368e1b57512b118e68080bd74a0862c76f5b38e9bafd9db9885
  Stored in directory: /root/.cache/pip/wheels/86/cd/9c/ad53b98a3eccf4ddeddccfce99d353781f206a1ac4287ca721
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993194 sha256=41cb400ed3692f278f91627d0a5d15ff6eda51a67d349544aaf182d9e30774c9
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d4

#Importing libraries

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf


In [3]:
tf.__version__

'2.4.0'

#Downloading the dataset

The dataset has been taken from Stanford University 

Movie Review Large Dataset

In [4]:
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git

Cloning into 'IMDB-Movie-Reviews-Large-Dataset-50k'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [5]:
data_train = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx', dtype = str)
data_train.head()

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg


In [6]:
data_test = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx', dtype = str)
data_test.head()

,Reviews,Sentiment
0,Who would have thought that a movie about a ma...,pos
1,After realizing what is going on around us ......,pos
2,I grew up watching the original Disney Cindere...,neg
3,David Mamet wrote the screenplay and made his ...,pos
4,"Admittedly, I didn't have high expectations of...",neg


In [7]:
(X_train,y_train),(X_test,y_test),preprocess= text.texts_from_df(train_df=data_train,
                   text_column='Reviews',
                   maxlen=400,
                   label_columns='Sentiment',
                   val_df=data_test,
                   preprocess_mode='bert')

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [8]:
X_train[0].shape

(25000, 400)

In [9]:
model=text.text_classifier(name='bert', 
                           train_data=(X_train,y_train),
                           preproc=preprocess)

                      

Is Multi-Label? False
maxlen is 400
done.


In [10]:
# Get learning rate
learner=ktrain.get_learner(model=model,
                           train_data=(X_train,y_train),
                            val_data=(X_test,y_test),
                           batch_size=6)

In [ ]:
# find out best learning rate?
# learner.lr_find()
# learner.lr_plot()

# it may take days or many days to find out.
#Optimal learning rate for this model = 2e-5

In [11]:
#Essentially fit is a very basic training loop, whereas fit one cycle uses the one cycle policy callback

learner.fit_onecycle(lr = 2e-5, epochs = 1)



begin training using onecycle policy with max lr of 2e-05...
4167/4167 [==============================] - ETA: 0s - loss: 0.3433 - accuracy: 0.8466

KeyboardInterrupt: ignored

In [13]:
predictor = ktrain.get_predictor(learner.model, preprocess)


In [14]:
#sample dataset to test on

data = ['this movie was horrible, the plot was really boring. acting was okay',
        'the fild is really sucked. there is not plot and acting was bad',
        'what a beautiful movie. great plot. acting was good. will see it again']


In [15]:
predictor.predict(data)

['neg', 'neg', 'pos']

In [16]:
predictor.save('/content/bert')